# Apache Spark: o que é e como utilizá-lo com o Google Colab

### Instalação e configuração das bibliotecas e suas dependências

In [ ]:
# Baixando e instalando o Java (JDK)
!apt-get install openjdk-8-jdk-headless -qq > /dev/null #Java

# Baixando e instalando o ecossistema Hadoop com o Spark
!wget -q https://downloads.apache.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7-hive1.2.tgz
!tar xf spark-3.0.1-bin-hadoop2.7-hive1.2.tgz

# Importando as bibliotecas Python para executar o Spark
!pip install -q findspark
!pip install -q pyspark

### Exploração das estruturas de pastas criadas

In [ ]:
# Java
!ls /usr/lib/jvm/

default-java		   java-11-openjdk-amd64     java-8-openjdk-amd64
java-1.11.0-openjdk-amd64  java-1.8.0-openjdk-amd64


In [ ]:
# Spark
!ls /content/

sample_data			       spark-3.0.1-bin-hadoop2.7-hive1.2.tgz.1
spark-3.0.1-bin-hadoop2.7-hive1.2      spark-3.0.1-bin-hadoop2.7-hive1.2.tgz.2
spark-3.0.1-bin-hadoop2.7-hive1.2.tgz  spark-3.0.1-bin-hadoop2.7-hive1.2.tgz.3


### Criação das variáveis de ambiente

In [ ]:
import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop2.7-hive1.2"

### Inicialização do Spark

In [ ]:
import findspark

findspark.init()

### Inicialização de uma sessão no Spark

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
spark

### Exploração do conjunto de dados Anscombe (um dos exemplos no Google Colab)

Quarteto de Anscombe é o nome dado a quatro conjuntos de dados que têm estatísticas descritivas quase idênticas (como a média e a variância), mas que têm distribuições muito diferentes e aparências muito distintas quando exibidos graficamente. Cada conjunto de dados consiste de onze pontos (x,y). Eles foram construídos em 1973 pelo estatístico Francis Anscombe, com o objetivo de demonstrar tanto a importância de se visualizar os dados antes de analisá-los, quanto o efeito dos outliers e outras observações influentes nas propriedades estatísticas. Ele descreveu o artigo como tendo a finalidade de combater a impressão entre os estatísticos de que "cálculos numéricos são exatos, mas gráficos são aproximados/grosseiros."

<center><img src="https://upload.wikimedia.org/wikipedia/commons/e/ec/Anscombe%27s_quartet_3.svg" width="800"></center>

In [ ]:
# Lendo o conjunto de dados
dataset = spark.read.format("json").option("multiLine", True).load("sample_data/anscombe.json")

# Apresentando as colunas
dataset.columns

['Series', 'X', 'Y']

In [ ]:
# Apresentando os primeiros 10 registros do conjunto de dados 
dataset.show(10)

+------+----+-----+
|Series|   X|    Y|
+------+----+-----+
|     I|10.0| 8.04|
|     I| 8.0| 6.95|
|     I|13.0| 7.58|
|     I| 9.0| 8.81|
|     I|11.0| 8.33|
|     I|14.0| 9.96|
|     I| 6.0| 7.24|
|     I| 4.0| 4.26|
|     I|12.0|10.84|
|     I| 7.0| 4.81|
+------+----+-----+
only showing top 10 rows



In [ ]:
# Apresentando o schema (tipos das variáveis) inferido pelo Spark
dataset.printSchema()

root
 |-- Series: string (nullable = true)
 |-- X: double (nullable = true)
 |-- Y: double (nullable = true)



In [ ]:
# Importando a biblioteca de funções SQL do Spark
from pyspark.sql import functions as F

# Agrupando os dados pelo tipo de conjunto, calculando os valores aggregados (médias) de X e Y e ordenando pelo tipo de conjunto
dataset_agrupado = dataset.groupBy("Series") \
                          .agg(F.avg("X").alias("X_agrupado"), F.avg("Y").alias("Y_agrupado"))  \
                          .orderBy("Series")

# Apresentando o resultado do agrupamento
dataset_agrupado.show()

+------+----------+-----------------+
|Series|X_agrupado|       Y_agrupado|
+------+----------+-----------------+
|     I|       9.0|              7.5|
|    II|       9.0|7.500909090909091|
|   III|       9.0|7.500000000000001|
|    IV|       9.0| 7.50090909090909|
+------+----------+-----------------+



In [ ]:
# Explicação do plano de execução do Spark
dataset_agrupado.explain()

== Physical Plan ==
*(3) Sort [Series#0 ASC NULLS FIRST], true, 0
+- Exchange rangepartitioning(Series#0 ASC NULLS FIRST, 200), true, [id=#62]
   +- *(2) HashAggregate(keys=[Series#0], functions=[avg(X#1), avg(Y#2)])
      +- Exchange hashpartitioning(Series#0, 200), true, [id=#58]
         +- *(1) HashAggregate(keys=[Series#0], functions=[partial_avg(X#1), partial_avg(Y#2)])
            +- FileScan json [Series#0,X#1,Y#2] Batched: false, DataFilters: [], Format: JSON, Location: InMemoryFileIndex[file:/content/sample_data/anscombe.json], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<Series:string,X:double,Y:double>




### Referências:

https://movile.blog/introducao-a-spark-usando-o-google-colab/

https://pt.wikipedia.org/wiki/Quarteto_de_Anscombe